In [1]:
# reload magics
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datasets
from project_modules.io import load_dataset_to_df
from project_modules.classifcation import classify_MP,getXY, boruta_fs
from project_modules.utils import MPutils
from sklearn.ensemble import RandomForestClassifier
import multiprocessing
from sklearn.model_selection import cross_val_score
import cupy as cp
from datetime import datetime

from tqdm.notebook import tqdm

# from project_modules.utils import get_logger
# logger = get_logger("log-data-combine-split.log")
# # read the parameter file

# from project_modules.utils import read_parameters
# parms = read_parameters("/Users/david/projects/lc-project-data/project.yaml")

In [2]:
from pathlib import Path

# input_path = Path("../lc-project-data")
# output_path = Path("../lc-project-data")

# for MP,
# input_path = Path("../Data/DataV3")
# output_path = Path("OUTPUT/MP/05-classifiers/DataV3/")

In [3]:
lScorersBinary = [
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    # "average_precision",
    "NPV",
    "PPV",
    # "neg_mean_squared_error",
]
lResCol = [
    "Title",
    "cv",
    "param_clf",
    "param_clf__max_depth",
    "param_clf__n_estimators",
    "param_clf__random_state",
    # "param_clf__max_iter",
    "mean_test_accuracy",
    "mean_test_balanced_accuracy",
    "mean_test_roc_auc",
    "mean_test_f1",
    "mean_test_recall",
    "mean_test_sensitivity",
    "mean_test_specificity",
    "mean_test_precision",
    "mean_test_NPV",
    "mean_test_PPV",
]

lPrettyCols = [
    "MainDataset",
    "RunType",
    "classifier",
    "brt_nTrue",
    "brt_nTop",
    "accuracy",
    "balanced_accuracy",
    "roc_auc",
    "f1",
    "recall",
    "sensitivity",
    "specificity",
    "precision",
    "NPV",
    "PPV",
    "brt_md",
]

In [4]:
# lDataNames = ['T81','T85','DT']
lDataNames = [
    # "dfcmplt", # Done
    # "dfcmplt_SITE1", # Done
    # "dfcmplt_SITE2", # Done
    # "dfcmplt_SITE3", # Done
    # "dfcmplt_SITE4", # Done
    "dfcmplt_SITE5",  # LEFT
    "dfcmplt_SITE6",  # LEFT
    "dfcmplt_SITE7",  # LEFT
]
dataDir = "../Data/DataV3/TTS/"

In [5]:
# saveDir = MPutils.get_saving_dir(f"OUTPUT/MP/05-classifiers/DataV2/{dataName}/")


lClassifiers = ["RF", "LogR",'XGBcpu'] # different classifiers #NOTE - XGBcpu is faster than XGBcuda for some reason?
lCV = [5] # different cross-validation splits
lTrees=[100,1000] #num trees
lMD=[3,6,10,15, None] #max depth

lBrt_MD = [3, 5, 7]
brt_trees = 1000
brt_iter = 500
brt_thres = 100
brt_toprank = 5


# FIXME - BELOW CODE FOR TESTING PURPOSES
# dataName = 'dfcmplt'
# lClassifiers = [
#     "RF",
#     "LogR",
#     "XGBcpu",
# ]  # different classifiers #NOTE - XGBcpu is faster than XGBcuda for some reason?
# lCV = [5]  # different cross-validation splits
# lTrees = [10]  # num trees
# lMD = [3]  # max depth

# lBrt_MD = [3]
# brt_trees=10
# brt_iter = 10
# brt_thres=100
# brt_toprank=5

# Automated ML BRT and testing

In [6]:
lRunDataFrames = []
for dataName in tqdm(lDataNames,desc='DataSet Main Outer Loop'):

    df_tr = load_dataset_to_df(f"{dataDir}{dataName}_Train.arrow", verbose=True)
    df_ts = load_dataset_to_df(f"{dataDir}{dataName}_Test.arrow", verbose=True)
    df_h = load_dataset_to_df(f"{dataDir}{dataName}_Holdout.arrow", verbose=True)
    df_full = pd.concat([df_tr,df_ts,df_h])
    lColDrop = ["__index_level_0__", "LC_STATUS_SITE",'SITE']

    for df in [df_tr,df_ts,df_h,df_full]:
        df.drop(df[df["LC_STATUS"] == 2].index, inplace=True)  # drop HC
        ##Depreciated below based on doing this in the splice data
        # df["LC_STATUS"] = df["LC_STATUS"].apply(
        #     lambda x: 1 if x == "LC_POS" else 0
        # )  # Convert to 0==LCNeg, 1==LCPos
        df.drop(columns=lColDrop, inplace=True)  # drop unneeded columns

    saveDir = MPutils.get_saving_dir(
        f"OUTPUT/MP/05-classifiers/DataV3/{dataName}/"
    )

    ## HOLDOUT SET ONLY
    x_h, y_h = getXY(df_h)

    dfRHold = classify_MP(
        X=x_h,
        y=y_h,
        lScorers=lScorersBinary,
        lClassifiers=lClassifiers,
        lTrees=lTrees,
        lMD=lMD,
        lCV=lCV,
    )
    dfRHold["Title"] = f"{dataName}_Holdout"
    dfRHold['RunType']='Holdout'

    dfRHold.to_csv(f"{saveDir}CA_{dataName}_Holdout.csv", index=False)

    ## Full SET ONLY
    x_F, y_F = getXY(df_full)

    dfRFull = classify_MP(
        X=x_F,
        y=y_F,
        lScorers=lScorersBinary,
        lClassifiers=lClassifiers,
        lTrees=lTrees,
        lMD=lMD,
        lCV=lCV,
    )
    dfRFull["Title"] = f"{dataName}_Full"
    dfRFull["RunType"] = "Full"

    dfRFull.to_csv(f"{saveDir}CA_{dataName}_Full.csv", index=False)

    ## BORUTA Feature Selection

    x_Tr, y_Tr = getXY(df_tr)
    lBrtResults = []
    for boruta_md in tqdm(lBrt_MD[:],desc='Boruta MD Loop'):
        # run brouta = get true and top feat
        trueFeat, topFeat = boruta_fs(
            X=x_Tr.values,
            y=y_Tr,
            feat_list=x_Tr.columns,
            trees=brt_trees,
            ittr=brt_iter,
            threshold=brt_thres,
            top_rank=brt_toprank,
            verbose=0,
            model=RandomForestClassifier(
                n_jobs=-1, class_weight="balanced", max_depth=boruta_md, random_state=42
            ),
            fileName=f"{saveDir}FS_{dataName}_Boruta_T{brt_trees}_itrr{brt_iter}_th{brt_thres}_topR{brt_toprank}_MD{boruta_md}.csv",
        )

        ## Test Brouta Features
        x_TsMain, y_Ts = getXY(df_ts)

        x_Ts_True = x_TsMain[trueFeat]
        x_Ts_Top = x_TsMain[topFeat]

        # True Feat
        if len(trueFeat) > 0:
            dfR_True = classify_MP(
                X=x_Ts_True,
                y=y_Ts,
                lScorers=lScorersBinary,
                lClassifiers=lClassifiers,
                lTrees=lTrees,
                lMD=lMD,
                lCV=lCV,
            )
            dfR_True["Title"] = f"{dataName}_Boruta_True"
            dfR_True["brt_md"] = boruta_md
            dfR_True['brt_nTrue'] = len(trueFeat)
            dfR_True['brt_nTop'] = len(topFeat)
            dfR_True["RunType"] = "Boruta_True"
            lBrtResults.append(dfR_True)

        # Top Feat
        if len(topFeat) > 0:
            dfR_Top = classify_MP(
                X=x_Ts_Top,
                y=y_Ts,
                lScorers=lScorersBinary,
                lClassifiers=lClassifiers,
                lTrees=lTrees,
                lMD=lMD,
                lCV=lCV,
            )
            dfR_Top["Title"] = f"{dataName}_Boruta_Top"

            dfR_Top["brt_md"] = boruta_md
            dfR_Top['brt_nTrue'] = len(trueFeat)
            dfR_Top["brt_nTop"] = len(topFeat)
            dfR_Top["RunType"] = "Boruta_Top"
            lBrtResults.append(dfR_Top)

    dfCLFRun = pd.concat([dfRHold, dfRFull]+lBrtResults)

    dfCLFRun["MainDataset"] = dataName
    dfCLFRun["date"] = datetime.today()
    dfCLFRun["brt_params"] = (
        f"T{brt_trees}_itrr{brt_iter}_th{brt_thres}_topR{brt_toprank}"
    )

    dfCLFRun = MPutils.reorder_columns(dfCLFRun, ["MainDataset", "Title", "date",'brt_nTrue','brt_nTop'])

    dfCLFRun.to_csv(f"{saveDir}CLFRun_{dataName}_Results.csv", index=False)
    lRunDataFrames.append(dfCLFRun)

DataSet Main Outer Loop:   0%|          | 0/8 [00:00<?, ?it/s]

Boruta MD Loop:   0%|          | 0/3 [00:00<?, ?it/s]

Boruta MD Loop:   0%|          | 0/3 [00:00<?, ?it/s]

Boruta MD Loop:   0%|          | 0/3 [00:00<?, ?it/s]

Boruta MD Loop:   0%|          | 0/3 [00:00<?, ?it/s]

Boruta MD Loop:   0%|          | 0/3 [00:00<?, ?it/s]

/home/mtrx/miniconda3/envs/clust/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mtrx/miniconda3/envs/clust/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mtrx/miniconda3/envs/clust/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/m

Boruta MD Loop:   0%|          | 0/3 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
dfCombRes = pd.concat(lRunDataFrames)

In [8]:
dfCombRes["classifier"] = dfCombRes["param_clf"].apply(lambda x: x.__str__().split("(")[0])

In [9]:
lColFirst = [
    "MainDataset",
    "Title",
    "date",
    "classifier",
    'RunType',
    "cv",
    "brt_nTrue",
    "brt_nTop",
]

In [10]:
dfCombRes = MPutils.reorder_columns(dfCombRes, lColFirst)

In [11]:
dfCombRes.to_csv('OUTPUT/MP/05-classifiers/DataV3/CLFRunCombined.csv',index=False)

In [12]:
lColsSel = lColFirst + [
    x
    for x in dfCombRes
    if (x.startswith("mean_test") or x.startswith("param_") or x.startswith("brt_"))
    and  x not in lColFirst
]

In [13]:
dfCombRes[lColsSel].to_csv(
    "OUTPUT/MP/05-classifiers/DataV3/CLFRunCombined_SelCol.csv", index=False
)

0.93

# Holdoutset - validation testing - Boruta Tested Best Feature + Sites

In [91]:
# lDT_7 = MPutils.getTrueFeatList(
#     "OUTPUT/MP/05-classifiers/DataV2/DT/FS_DT_Boruta_T1000_itrr500_th100_topR5_MD7.csv"


lCmplt = MPutils.getTrueFeatList(
    "OUTPUT/MP/05-classifiers/DataV2/T81/FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD5.csv"
)


lColFS = lCmplt
# dataName = "T81"
selFeatName = "FS_dfcmplt_Boruta_T1000_itrr500_th100_topR5_MD5"

In [ ]:
lRunDataFrames = []
for dataName in tqdm(lDataNames,desc='DataSet Main Outer Loop'):

    df_h = load_dataset_to_df(f"{dataDir}{dataName}_Holdout.arrow", verbose=True)

    lColDrop = ["__index_level_0__", "LC_STATUS_SITE",'SITE']

    df_h.drop(df_h[df_h["LC_STATUS"] == 2].index, inplace=True)  # drop HC

    # df_h["LC_STATUS"] = df_h["LC_STATUS"].apply(
    #     lambda x: 1 if x == "LC_POS" else 0
    # )  # Convert to 0==LCNeg, 1==LCPos
    df_h.drop(columns=lColDrop, inplace=True)  # drop unneeded columns

    saveDir = MPutils.get_saving_dir(
        f"OUTPUT/MP/05-classifiers/DataV3/{dataName}/"
    )

    ## HOLDOUT SET ONLY
    x_h, y_h = getXY(df_h)
    x_h = x_h[lColFS]

    dfRHold = classify_MP(
        X=x_h,
        y=y_h,
        lScorers=lScorersBinary,
        lClassifiers=lClassifiers,
        lTrees=lTrees,
        lMD=lMD,
        lCV=lCV,
    )
    dfRHold["Title"] = f"{dataName}_HoldoutVal"
    dfRHold['RunType']='HoldoutVal'
    dfRHold["FeatureSet"] = selFeatName
    dfRHold['date']=datetime.today()
    dfRHold["classifier"] = dfRHold["param_clf"].apply(lambda x: x.__str__().split("(")[0])
    dfRHold["MainDataset"] = dataName

    lColFirst = [
        "MainDataset",
        "Title",
        "date",
        "classifier",
        "RunType",
        "cv",
    ]
    dfRHold = MPutils.reorder_columns(dfRHold, lColFirst)

    ## Export
    dfRHold.to_csv(f"{saveDir}Val_{dataName}_HoldoutVal_{selFeatName}.csv", index=False)

    ## Select Columns
    lColsSel = lColFirst + [
        x
        for x in dfRHold
        if (x.startswith("mean_test") or x.startswith("param_") or x.startswith("brt_"))
        and x not in lColFirst
    ]

    dfRHold[lColsSel].to_csv(
        f"{saveDir}Val_{dataName}_HoldoutVal_{selFeatName}_SelCol.csv",
        index=False,
    )

    lRunDataFrames.append(dfRHold)

    ### Feature Importance

    clf = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=42)
    clf.fit(x_h, y_h)
    feature_importances = clf.feature_importances_
    feature_importances = pd.Series(feature_importances, index=x_h.columns).sort_values(
        ascending=False
    )

    dfFeatureImp = pd.DataFrame(feature_importances).reset_index()
    dfFeatureImp.columns = ["Feature", "Importance"]
    dfFeatureImp["Importance"] = dfFeatureImp["Importance"].round(4)
    dfFeatureImp["Rank"] = dfFeatureImp.rank(ascending=False)["Importance"].astype(int)
    dfFeatureImp = dfFeatureImp[["Rank", "Feature", "Importance"]]

    dfFeatureImp.to_csv(
        f"{saveDir}FR_{dataName}_HoldoutVal{selFeatName}.csv", index=False
    )


dfCombRes = pd.concat(lRunDataFrames)

dfCombRes = MPutils.reorder_columns(dfCombRes, lColFirst)

dfCombRes.to_csv(
    "OUTPUT/MP/05-classifiers/DataV3/CLFHoldoutValRunCombined_{selFeatName}.csv",
    index=False,
)
lColsSel = lColFirst + [
    x
    for x in dfCombRes
    if (x.startswith("mean_test") or x.startswith("param_") or x.startswith("brt_"))
    and x not in lColFirst
]
dfCombRes[lColsSel].to_csv(
    "OUTPUT/MP/05-classifiers/DataV3/CLFHoldoutValRunCombined_{selFeatName}_selCol.csv",
    index=False,
)

In [ ]:
# df_h = load_dataset_to_df(f"{dataDir}{dataName}_Holdout.arrow", verbose=True)

# lColDrop = ["__index_level_0__", "LC_STATUS_SITE",'SITE']


# df_h.drop(df_h[df_h["LC_STATUS"] == 2].index, inplace=True)  # drop HC

# # df_h["LC_STATUS"] = df_h["LC_STATUS"].apply(
# #     lambda x: 1 if x == "LC_POS" else 0
# # )  # Convert to 0==LCNeg, 1==LCPos
# df_h.drop(columns=lColDrop, inplace=True)  # drop unneeded columns

# saveDir = MPutils.get_saving_dir(
#     f"OUTPUT/MP/05-classifiers/DataV3/{dataName}/"
# )

# ## HOLDOUT SET ONLY
# x_h, y_h = getXY(df_h)
# x_h = x_h[lColFS]

# dfRHold = classify_MP(
#     X=x_h,
#     y=y_h,
#     lScorers=lScorersBinary,
#     lClassifiers=lClassifiers,
#     lTrees=lTrees,
#     lMD=lMD,
#     lCV=lCV,
# )
# dfRHold["Title"] = f"{dataName}_HoldoutVal"
# dfRHold['RunType']='HoldoutVal'
# dfRHold["FeatureSet"] = selFeatName
# dfRHold['date']=datetime.today()
# dfRHold["classifier"] = dfRHold["param_clf"].apply(lambda x: x.__str__().split("(")[0])
# dfRHold["MainDataset"] = dataName

# lColFirst = [
#     "MainDataset",
#     "Title",
#     "date",
#     "classifier",
#     "RunType",
#     "cv",
# ]
# dfRHold = MPutils.reorder_columns(dfRHold, lColFirst)

# dfRHold.to_csv(f"{saveDir}Val_{dataName}_HoldoutVal_{selFeatName}.csv", index=False)

In [ ]:
# dfCombRes = pd.concat(lRunDataFrames)
# dfCombRes["classifier"] = dfCombRes["param_clf"].apply(
#     lambda x: x.__str__().split("(")[0]
# )
# lColFirst = [
#     "MainDataset",
#     "Title",
#     "date",
#     "classifier",
#     "RunType",
#     "cv",
#     "brt_nTrue",
#     "brt_nTop",
# ]
# dfCombRes = MPutils.reorder_columns(dfCombRes, lColFirst)
# dfCombRes.to_csv("OUTPUT/MP/05-classifiers/DataV3/CLFRunCombined.csv", index=False)
# lColsSel = lColFirst + [
#     x
#     for x in dfCombRes
#     if (x.startswith("mean_test") or x.startswith("param_") or x.startswith("brt_"))
#     and x not in lColFirst
# ]
# dfCombRes[lColsSel].to_csv(
#     "OUTPUT/MP/05-classifiers/DataV3/CLFRunCombined_SelCol.csv", index=False
# )

In [ ]:
# lColsSel = lColFirst + [
#     x
#     for x in dfRHold
#     if (x.startswith("mean_test") or x.startswith("param_") or x.startswith("brt_"))
#     and x not in lColFirst
# ]
# dfRHold[lColsSel].to_csv(f"{saveDir}Val_{dataName}_HoldoutVal_{selFeatName}_SelCol.csv", index=False)

### Feature Importance of Best Set


In [ ]:
clf = RandomForestClassifier(n_estimators=1000,max_depth=10,random_state=42)
clf.fit(x_h,y_h)
feature_importances = clf.feature_importances_
feature_importances = pd.Series(feature_importances, index=x_h.columns).sort_values(
    ascending=False
)

In [ ]:
dfFeatureImp = pd.DataFrame(feature_importances).reset_index()
dfFeatureImp.columns = ['Feature','Importance']
dfFeatureImp["Importance"] = dfFeatureImp["Importance"].round(4)
dfFeatureImp["Rank"] = dfFeatureImp.rank(ascending=False)["Importance"].astype(int)
dfFeatureImp = dfFeatureImp[['Rank','Feature','Importance']]

In [ ]:
dfFeatureImp

,Rank,Feature,Importance
0,1,SYMPT-fatigue___2,0.2420
1,2,SYMPT-shortness_of_breath_dyspne___2,0.1566
2,3,SYMPT-loss_of_taste_lost_of_smel___2,0.0913
3,4,SYMPT-cough___2,0.0859
4,5,SYMPT-dizziness___2,0.0811
5,6,SYMPT-joint_pain_arthralgia___2,0.0685
6,7,SYMPT-headache___2,0.0533
7,8,SYMPT-muscle_aches_myalgia___2,0.0475
8,9,age,0.0324
9,10,SYMPT-chest_pain___2,0.0274


In [ ]:
dfFeatureImp.to_csv(f"{saveDir}FR_{dataName}_HoldoutVal{selFeatName}.csv", index=False)

In [ ]:
clf = RandomForestClassifier(n_estimators=1000, max_depth=None, random_state=42)
cross_val_score(clf, x_h, y_h, cv=5, scoring="accuracy").mean().round(2)

0.93

# Holdoutset - validation testing PER SITE - Boruta Tested Best Featuref

In [26]:
# site_name_dict = {
#                   1: 'LDN',
#                   2: 'MTL',
#                   3: 'SAN',
#                   4: 'RIO',
#                   5: 'LUS',
#                   6:'CA',
#                   7:'NorthA'
#                 }

In [27]:
# # lDT_7 = MPutils.getTrueFeatList(
# #     "OUTPUT/MP/05-classifiers/DataV2/DT/FS_DT_Boruta_T1000_itrr500_th100_topR5_MD7.csv"
# # )


# # lColFS = lDT_7
# # dataName = "DT"
# # selFeatName = "FS_DT_Boruta_T1000_itrr500_th100_topR5_MD7"

# # - DT BRT_MD7 True has the best test numbers with 96%.
# # - BUT the DT dataset has basically no LUS patients…. thus is not a good representation of a “global cohort”
# # - THUS we should probably use and report the T81 dataset

# lR81_5 = MPutils.getTrueFeatList(
#     "OUTPUT/MP/05-classifiers/DataV2/T81/FS_T81_Boruta_T1000_itrr500_th100_topR5_MD5.csv"
# )


# lColFS = lR81_5
# dataName = "T81"
# selFeatName = "FS_T81_Boruta_T1000_itrr500_th100_topR5_MD5"

In [28]:
# lSiteRes = []
# for i in tqdm(site_name_dict.keys()):
#     df_h = load_dataset_to_df(f"{dataDir}{dataName}_Holdout.arrow", verbose=True)
#     if i==7:
#         df_h = df_h[df_h['SITE']<=3]
#     elif i==6:
#         df_h = df_h[df_h['SITE']<=2]
#     else:
#         df_h = df_h[df_h["SITE"] ==i]

#     lColDrop = ["__index_level_0__", "LC_STATUS_SITE",'SITE']

#     df_h.drop(df_h[df_h["LC_STATUS"] == "HC"].index, inplace=True)  # drop HC
#     df_h["LC_STATUS"] = df_h["LC_STATUS"].apply(
#         lambda x: 1 if x == "LC_POS" else 0
#     )  # Convert to 0==LCNeg, 1==LCPos
#     df_h.drop(columns=lColDrop, inplace=True)  # drop unneeded columns

#     saveDir = MPutils.get_saving_dir(
#         f"OUTPUT/MP/05-classifiers/DataV2/{dataName}/SITE_{i}_{site_name_dict[i]}/"
#     )
#     ## HOLDOUT SET ONLY
#     x_h, y_h = getXY(df_h)
#     x_h = x_h[lColFS]

#     dfRHold = classify_MP(
#         X=x_h,
#         y=y_h,
#         lScorers=lScorersBinary,
#         lClassifiers=lClassifiers,
#         lTrees=lTrees,
#         lMD=lMD,
#         lCV=lCV,
#     )
#     dfRHold["Title"] = f"{dataName}_HoldoutVal"
#     dfRHold["RunType"] = "HoldoutVal_Site"
#     dfRHold["Site"] = site_name_dict[i]
#     dfRHold["FeatureSet"] = selFeatName
#     dfRHold["date"] = datetime.today()
#     dfRHold["classifier"] = dfRHold["param_clf"].apply(lambda x: x.__str__().split("(")[0])
#     dfRHold["MainDataset"] = dataName

#     lColFirst = [
#         "date",
#         "MainDataset",
#         "Title",
#         "RunType",
#         'Site',
#         "classifier",
#         "cv",
#     ]
#     dfRHold = MPutils.reorder_columns(dfRHold, lColFirst)

#     dfRHold.to_csv(
#         f"{saveDir}Val_{dataName}_HoldoutVal_{selFeatName}_SITE{i}.csv", index=False
#     )

#     lColsSel = lColFirst + [
#         x
#         for x in dfRHold
#         if (x.startswith("mean_test") or x.startswith("param_") or x.startswith("brt_"))
#         and x not in lColFirst
#     ]
#     dfRHold[lColsSel].to_csv(
#         f"{saveDir}Val_{dataName}_HoldoutVal_{selFeatName}_SITE{i}_SelCol.csv",
#         index=False,
#     )
#     lSiteRes.append(dfRHold)

#     ### Feature Importance

#     clf = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=42)
#     clf.fit(x_h, y_h)
#     feature_importances = clf.feature_importances_
#     feature_importances = pd.Series(feature_importances, index=x_h.columns).sort_values(
#         ascending=False
#     )

#     dfFeatureImp = pd.DataFrame(feature_importances).reset_index()
#     dfFeatureImp.columns = ["Feature", "Importance"]
#     dfFeatureImp["Importance"] = dfFeatureImp["Importance"].round(4)
#     dfFeatureImp["Rank"] = dfFeatureImp.rank(ascending=False)["Importance"].astype(int)
#     dfFeatureImp = dfFeatureImp[["Rank", "Feature", "Importance"]]

#     dfFeatureImp.to_csv(
#         f"{saveDir}FR_{dataName}_HoldoutVal{selFeatName}_SITE{i}.csv", index=False
#     )

  0%|          | 0/7 [00:00<?, ?it/s]

/home/mtrx/miniconda3/envs/clust/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mtrx/miniconda3/envs/clust/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mtrx/miniconda3/envs/clust/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/m

In [31]:
pd.concat(lSiteRes).to_csv('OUTPUT/MP/05-classifiers/DataV2/CLFRunCombined_Site.csv',index=False)

In [32]:
pd.concat(lSiteRes)[lColsSel].to_csv(
    "OUTPUT/MP/05-classifiers/DataV2/CLFRunCombined_Site_SelCol.csv", index=False
)